In [4]:
from chromadb import PersistentClient
import json
import os
import pandas as pd
from mistralai.client import MistralClient

MISTRAL_API_KEY = "hzfGrqIKAuQlijVjeRaCs4s0yEpgYgn8"

C:\Users\jules\AppData\Local\Temp\ipykernel_10200\2763690786.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
chromadb = PersistentClient(path='./chromadb')
collection = chromadb.get_or_create_collection(
    name=f"reviews"
)


In [6]:
mistral = MistralClient(MISTRAL_API_KEY)

In [36]:
def embed_batch(client:MistralClient,reviews_list : list):
    if len(reviews_list) == 0:
        return []
    if len(reviews_list) > 20:
        raise ValueError("Batch size too large")
    embeddings_response = client.embeddings(
        model="mistral-embed",
        input=reviews_list
    )
    embeddings = []
    for emb in embeddings_response.data:
        embeded = emb.embedding
        embeddings.append(embeded)
    return embeddings

In [8]:
def load_reviews():
    csv= pd.read_csv("../data/Sample_Books_rating.csv")
    return csv


In [9]:
data = load_reviews()

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)



In [11]:
ids = train['Id'].tolist()
texts = train['revue/texte'].tolist()
metadata = [{'title': title , 'score':score} for title,score in zip(train['Titre'], train['revue/score'])]


In [12]:
ids

['B0006BNTL0',
 'B000IN24Y4',
 '0061090867',
 '0130309737',
 '0783893310',
 'B0006AOQTU',
 'B000OVPFHS',
 'B000FJGQ2C',
 '0674011813',
 'B000NPRINY',
 '1417616962',
 'B000I1DFJ4',
 'B000BK39GG',
 'B000Q3EKV6',
 'B000J521DU',
 'B0007DXWTQ',
 'B0006X9ZAC',
 'B000BOCL3O',
 'B0008CH422',
 'B000NWUHR6',
 '0911560513',
 'B000H3MM06',
 '0553575171',
 'B000OVDPHA',
 'B0007JFFPE',
 'B000FAIRN2',
 '059527675X',
 'B000MUQ2KA',
 'B000L1T9NC',
 '0788191225',
 '0451194489',
 'B0007EEVUY',
 'B000FGGN1E',
 '0880880600',
 '1847022189',
 'B0007C10MS',
 'B0006SHMQ6',
 'B000P0VS56',
 'B0006CGMRC',
 '0671017225',
 '0743249976',
 '0451199669',
 '1551521873',
 'B000OVTKTC',
 'B000NRCOY0',
 'B000H468CI',
 '1581807708',
 '0743219465',
 'B000FC2QGE',
 '0971901767',
 '1592281001',
 '0764153269',
 '0471979317',
 '0786227834',
 'B000MOOAJG',
 'B0006AQZSU',
 'B000PD8REI',
 'B000BL3NZ2',
 'B0002XH6T8',
 'B0009K761M',
 '079104520X',
 '0689823568',
 '0806526114',
 'B000PHRL0U',
 'B000H9K9LO',
 '0275969533',
 'B000NDSX

In [13]:
texts

['As a foreign student,I was put into the flood of words',
 "Most biographies are boring, but this one is not! Highly entertaining and historically informative, this book is a must for anyone interested in Horowitz's art and life. You won't put it down until you've finished reading it!",
 'As every other reviewer said, you can\'t put this one down and it\'s 858 pages! Matthews\' writing as usual is phenomenal. My only complaint is with the sci fi aspect of the story, a child with telekinesis talents which just do NOT exist, and I was surprised given Matthews\' obvious athiestic views in every one of his books. Plus he uses this as a "deus ex machina" to save some of his characters who would otherwise be dead if the child hadn\'t come to the miraculous rescue with her gift.However this book has LOTS of characters who are colorfully written. Some of them just fade out, not sure why they were there in the first place (many needed chapters given to an artist who paints a nude portrait of a

In [14]:
batchs = [texts[i:i+20] for i in range(0, len(texts), 20)]
tmp_ids = [ids[i:i+20] for i in range(0, len(ids), 20)]
tmp_metadata = [metadata[i:i+20] for i in range(0, len(metadata), 20)]
for batch_loop,ids_loop,metadata_loop in zip(batchs,tmp_ids,tmp_metadata):
    embeddings = embed_batch(mistral,batch_loop)
    collection.upsert(
            ids= ids_loop,
            embeddings= embeddings,
            metadatas= metadata_loop
    )
    print(f"Inserted {len(embeddings)} reviews")


['As a foreign student,I was put into the flood of words', "Most biographies are boring, but this one is not! Highly entertaining and historically informative, this book is a must for anyone interested in Horowitz's art and life. You won't put it down until you've finished reading it!", 'As every other reviewer said, you can\'t put this one down and it\'s 858 pages! Matthews\' writing as usual is phenomenal. My only complaint is with the sci fi aspect of the story, a child with telekinesis talents which just do NOT exist, and I was surprised given Matthews\' obvious athiestic views in every one of his books. Plus he uses this as a "deus ex machina" to save some of his characters who would otherwise be dead if the child hadn\'t come to the miraculous rescue with her gift.However this book has LOTS of characters who are colorfully written. Some of them just fade out, not sure why they were there in the first place (many needed chapters given to an artist who paints a nude portrait of a p

DuplicateIDError: Expected IDs to be unique, found duplicates of: 0141804459

['ahaga',
 'ahaga',
 'ahaga',
 'ahaga',
 'ahaga',
 'ahaga',
 'ahaga',
 'ahaga',
 'ahaga',
 'ahaga']

In [16]:
test[:1] 

,Id,Titre,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte
6252,0738200662,From Third World To World Class: The Future Of...,NaN,NaN,NaN,3/3,5.0,902448000,An insightful and engrossing read,As a student of international business and eco...


In [23]:
test_text = test[1:5]['revue/texte'].tolist()

In [18]:
embed_batch(mistral,test_text)

["As a student of international business and economics, I have been required to read a variety of business and investing texts. However, I have yet to read anything, be it a textbook or a practical business book, that covers the scope of Peter Marber's From Third World To World Class. Marber's thorough grasp of history and professional experience makes for a comprehensive, insightful and truly enjoyable read that is informative and highly original. As compared to other books that discuss Emerging Markets, such as the very general biography of emerging markets investing by Mark Mobius or the predictions of Jeffrey Garten's The Big Ten, Marber's work combines theory and practice, producing a book that tackles today's most controversial business issues with a refreshingly optimistic perspective. But perhaps what is most impressive about Marber's work is that he establishes a clear path for the reader to follow. Unlike other authors, Marber gives you a well-researched past! , present and f

[[-0.033935546875,
  0.03289794921875,
  0.037872314453125,
  0.01491546630859375,
  0.015625,
  0.024627685546875,
  0.0350341796875,
  -0.0306854248046875,
  -0.00024366378784179688,
  0.01384735107421875,
  -0.018768310546875,
  0.035186767578125,
  -0.017181396484375,
  -0.00200653076171875,
  -0.05035400390625,
  0.04937744140625,
  -4.470348358154297e-06,
  0.005950927734375,
  0.0323486328125,
  0.022430419921875,
  0.00525665283203125,
  -0.0066070556640625,
  -0.0418701171875,
  0.030914306640625,
  0.004852294921875,
  0.002330780029296875,
  -0.017852783203125,
  -0.050811767578125,
  -0.005764007568359375,
  0.009124755859375,
  0.0003936290740966797,
  -0.02716064453125,
  -0.0024433135986328125,
  0.01471710205078125,
  0.0018157958984375,
  -0.023345947265625,
  -0.01148223876953125,
  0.0004589557647705078,
  0.02496337890625,
  0.023895263671875,
  0.0011615753173828125,
  -0.0053558349609375,
  -0.01195526123046875,
  -0.008270263671875,
  0.02410888671875,
  -0.03558

In [24]:
search = collection.query(
    query_embeddings=embed_batch(mistral,test_text),
    n_results=5
)

["This book has no new insight; no special slant or interests other than the fact that the people sited were in exile from the 1930's, to today. There is nothing wrong with this book. Yet there is nothing to make it stand out from the crowd. Some of the people include Albert Einstein, Thomas Mann and Otto Preminger.Thomas Mann Eros and Literature", 'The new edition of the book is written in the same manner as the first. New techniques have been added and the old material have been rewritten in a more educational way.This combination makes this book a perfect choise for a good start for beginners and a beauty to study for non.', "This is a terrific book. It's got all the information you would ever need to be vegan/vegetarian and then some. We've worked our way through about 30 of the over 200 recipes without one disappointment yet. Easy to read, easy to follow recipes with commonly found ingredients. We're losing the extra weight we were carrying around without even trying.", "I just co

In [25]:
search

{'ids': [['B0006COUA8',
   '0029095026',
   '1586482661',
   '0965104257',
   'B000IYZSKA'],
  ['B000FJGQ2C', '0967605822', 'B000NRAXDY', '0966088816', '032302968X'],
  ['1551521873', 'B000LB5KX0', '0778801187', 'B0002OKAAY', '1558704612'],
  ['B0007FJEAK', '0345320735', 'B000QKT7JY', 'B000N63PU8', 'B000I1DFJ4']],
 'distances': [[0.3382103145122528,
   0.33857041597366333,
   0.34631913900375366,
   0.35851991176605225,
   0.3662392199039459],
  [0.25565579533576965,
   0.26071736216545105,
   0.26800107955932617,
   0.269788920879364,
   0.2822387218475342],
  [0.229910209774971,
   0.2602739930152893,
   0.26551899313926697,
   0.27553635835647583,
   0.3040226101875305],
  [0.24081876873970032,
   0.3020991086959839,
   0.32149407267570496,
   0.32430732250213623,
   0.33631432056427]],
 'metadatas': [[{'score': 3.0,
    'title': 'The Futurians: The story of the science fiction "family" of the 30\'s that produced today\'s top SF writers and editors'},
   {'score': 5.0,
    'title': 

In [42]:

def test_accuracy(collection, test:pd.DataFrame):
    results = []
    for tmp_text, tmp_score in zip(test['revue/texte'],test['revue/score']):
        search = collection.query(
            query_embeddings=embed_batch(mistral,[tmp_text]),
            n_results=5
        )
        results = search['metadatas']
        sum_score = 0
        for result in results:
            for tmp_result in result:
                sum_score += tmp_result['score']
        print(sum_score/5)
        print(tmp_score)
    

In [43]:
small_test = test[:5]
test_accuracy(collection, small_test)

4.8
5.0
3.6
3.0
4.6
5.0
4.8
5.0
1.6
3.0
